In [13]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import json
from tradingview_screener import Query, col
import rookiepy
from sklearn.preprocessing import MinMaxScaler
from gbm_optimizer import optimize_gbm, gbm


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

our_picks = ["UPST", "HOOD", "CHWY", "IBIT", "ASO", "DKNG", "PENN", "NDAQ", "SMCI", "TEM", "PIN", "HIMS"]



In [14]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [15]:
    
def screen_stocks():
    # Get cookies for TradingView session
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))
    
    _, df = Query().select('close','change', 'Perf.3M').where(
        col('close').between(20, 55),
        col('change').between(-4,-2),
        col('Perf.3M') > 0,
        col('exchange').isin(['AMEX', 'CBOE', 'NASDAQ', 'NYSE']),

        ).limit(1000).get_scanner_data(cookies=cookies)
    
    df[['exchange', 'ticker']] = df['ticker'].str.split(':', expand=True)
    
    return df


def get_rolling_price_change_avg(ticker: str, days: int):
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days+10)
        
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            return None, None
        
        data = data.sort_index()
        current_price = get_current_stock_price(ticker)

        data['Price_Change'] = ((current_price - data['Close'].shift(1)) / data['Close'].shift(1)) * 100

        rolling_avg = data['Price_Change'].rolling(window=min(days, len(data))).mean().iloc[-1]

        return rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None


def get_open_interest(api_key: str, secret_key: str, option_symbol: str):
    """Fetches open interest for a given option contract using Alpaca's Trading API."""
    url = f"https://paper-api.alpaca.markets/v2/options/contracts/{option_symbol}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data.get("open_interest")  # Extract OI if available
    except requests.exceptions.RequestException as e:
        print(f"Error fetching open interest for {option_symbol}: {e}")
        return None
    

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def get_option_chain(api_key: str, secret_key: str, ticker: str, expiration_date: datetime):
    expiration_str = expiration_date.strftime("%Y-%m-%d")  
    
    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{ticker}?feed=indicative&limit=100&expiration_date={expiration_str}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        option_chain = data.get("snapshots", {})

        if not option_chain:
            return None

        parsed_data = []
        for symbol, details in option_chain.items():
            expiration_start = len(symbol) - 15
            option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
            strike_price = int(symbol[expiration_start+7:]) / 1000  

            # open_interest = get_open_interest(api_key, secret_key, symbol)
            # open_interest = int(open_interest) if open_interest is not None else 0

            greeks = details.get("greeks", {}) or {}
            latest_quote = details.get("latestQuote", {})

            parsed_data.append({
                "symbol": ticker,
                "expiration_date": expiration_str,  
                "option_type": option_type,
                "strike_price": strike_price,
                "delta": greeks.get("delta"),
                "gamma": greeks.get("gamma"),
                "rho": greeks.get("rho"),
                "theta": greeks.get("theta"),
                "vega": greeks.get("vega"),
                "implied_volatility": details.get("impliedVolatility"),
                "ask_price": latest_quote.get("ap"),
                "ask_size": latest_quote.get("as"),
                "bid_price": latest_quote.get("bp"),
                "bid_size": latest_quote.get("bs"),
                # "open_interest": open_interest 

            })

        return pd.DataFrame(parsed_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None
    


def compute_score(contracts):
    """Compute the weighted score for contracts using normalized values."""
    temp_contracts = contracts.copy()
    
    scaler = MinMaxScaler()
    temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']] = scaler.fit_transform(
        temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']]
    )
    
    temp_contracts['score'] = (
        0.60 * temp_contracts['profitability_likelihood'] +
        0.40 * temp_contracts['return_percent'] 
        # 0.15 * temp_contracts['sortino_ratio']
    )
    
    contracts['score'] = temp_contracts['score']
    
    return contracts

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score while keeping original values."""
    contracts = compute_score(contracts)
    contracts = contracts.sort_values(by='score', ascending=False)
    return contracts

def compute_score(contracts):
    """Compute the weighted score for contracts using normalized values."""
    temp_contracts = contracts.copy()
    
    scaler = MinMaxScaler()
    # Note: use the new sortino_ratio in place of the sharpe_ratio
    temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']] = scaler.fit_transform(
        temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']]
    )
    
    temp_contracts['score'] = (
        0.60 * temp_contracts['profitability_likelihood'] +
        0.25 * temp_contracts['return_percent'] +
        0.15 * temp_contracts['sortino_ratio']
    )
    
    contracts['score'] = temp_contracts['score']
    return contracts

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score while keeping original values."""
    contracts = compute_score(contracts)
    contracts = contracts.sort_values(by='score', ascending=False)
    return contracts

def audit_contracts(expiration_date: datetime):
    daily_risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/252) - 1) * 100

    simulation_attempts = 500
    optimizer_training_period = "1y"
    bin_length = 18
    days_to_expiration = np.busday_count(datetime.today().date(), expiration_date.date())

    all_options = pd.DataFrame(columns=[
        'symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 
        'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 
        'ask_size', 'bid_price', 'bid_size', 'open_interest'
    ])
    
    candidates = screen_stocks()['ticker'].to_list()
    candidates.extend(our_picks)    

    for symbol in our_picks[0:1]:
        option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)

        if option_chain is None or option_chain.empty:
            continue 
        
        put_chain = option_chain[
            (option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())
        ].sort_values(by='strike_price', ascending=True)

        price = get_current_stock_price(symbol)
        optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length, simulation_attempts=simulation_attempts)

        # Run simulations once for all contracts
        simulated_prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, deltaT=days_to_expiration, dt=1, simulations=simulation_attempts)

        for index, contract in put_chain.iterrows():
            strike_price = contract['strike_price']
            premium_collected = (contract['bid_price'] + contract['ask_price']) / 2
            simulated_returns = []
            simulated_final_prices = []
            profitable_count = 0

            if (((contract['ask_price'] - contract['bid_price']) / contract['bid_price']) * 100 > 30) and (contract['open_interest'] > 50):
                continue  

            # Calculate profitability based on pre-generated simulations
            for i in range(simulation_attempts):
                final_price = simulated_prices[i, -1]
                if final_price >= strike_price:
                    profitable_count += 1
                    net_return = (premium_collected / strike_price) * 100
                else:
                    net_return = ((premium_collected - (strike_price - final_price)) / strike_price) * 100

                simulated_returns.append(net_return)
                simulated_final_prices.append(final_price)

            profitability_chance = (profitable_count / simulation_attempts) * 100
            percent_return = (premium_collected / strike_price) * 100  
            avg_return = np.mean(simulated_returns)
            avg_price = np.mean(simulated_final_prices)
            
            risk_free_return = daily_risk_free_rate * days_to_expiration
            downside_returns = [r for r in simulated_returns if r < risk_free_return]
            downside_std = np.std(downside_returns, ddof=1) if len(downside_returns) > 1 else 0

            if downside_std != 0:
                sortino_ratio = ((avg_return - risk_free_return) / downside_std) * np.sqrt(252 / days_to_expiration)
            else:
                sortino_ratio = 0

            put_chain.at[index, 'fill_price'] = premium_collected
            put_chain.at[index, 'current_price'] = price
            put_chain.at[index, 'final_price'] = avg_price
            put_chain.at[index, 'profitability_likelihood'] = profitability_chance
            put_chain.at[index, 'return_percent'] = percent_return
            put_chain.at[index, 'average_return'] = avg_return
            put_chain.at[index, 'sortino_ratio'] = sortino_ratio

        all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)

    return all_options


In [16]:

# priced_contracts = audit_contracts(expiration_date = datetime(year=2025, month=2, day=14))

import cProfile
import pstats
from datetime import datetime

def profile_audit_contracts():
    profiler = cProfile.Profile()
    profiler.enable()
    
    # Call the function you want to profile
    audit_contracts(expiration_date=datetime(year=2025, month=2, day=14))
    
    profiler.disable()
    
    # Sort by cumulative time and display everything
    stats = pstats.Stats(profiler)
    stats.strip_dirs().sort_stats("cumulative").print_stats()  # Sort by cumulative time

    print("\nTop calls sorted by time (self time only):")
    stats.strip_dirs().sort_stats("time").print_stats()  # Sort by time spent in the function itself

profile_audit_contracts()



TypeError: optimize_gbm() got an unexpected keyword argument 'simulation_attempts'

In [ ]:
contracts = select_optimal_contract(priced_contracts)
print('length',len(contracts))
display(contracts)

NameError: name 'priced_contracts' is not defined